## Importsetup

In [4]:
###########################################################################
from app.main.extraction.dxf_extraction_coordinator import DxfExtractionCoordinator
from app.main.models.wall import Wall 
from app.main.models.architecture_wall import Architecture_Wall
from app.main.models.point import Point 
from app.main.models.window import Window
from app.main.models.line import Line
from app.main.services.walls_extraction import WallsExtraction
from app.main.services.dxf_extraction_simple import DxfExtractionSimple
from app.main.dtos.request.extraction_request_file import ExtractionRequestFile
from app.main.models.line import Line 
from app.main.constants import (PLOT_DPI, PLOT_LINE_WIDTH_LARGE,
                                PLOT_LINE_WIDTH_SMALL, DefaultColors,
                                OrientationColors, PlotStyle, PlotTypes)
###########################################################################
from shapely.geometry.polygon import LinearRing
from shapely.geometry.polygon import Polygon as shapePolygon
from shapely.geometry import Point as shapePoint
from shapely.geometry import LineString
from shapely.geometry import Point as ShapelyPoint
import math
import numpy as np
import itertools as it
from scipy.spatial.distance import cdist
import ezdxf
import uuid

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import operator

###########################################################################
file_path_simple = '../../media/Einfache_Plaene_Compass/'
file_path_architecture = '../../media/Architektenplaene/'
###########################################################################
file_name_simple = 'R-Bau_D_EG.dxf'
file_name_architecutre = 'R-Bau_A_EG.dxf'
###########################################################################
simple_dxf = ezdxf.readfile(file_path_simple+file_name_simple)
architecture_dxf = ezdxf.readfile(file_path_architecture+file_name_architecutre)
simple_model = simple_dxf.modelspace()
arichtecture_model = architecture_dxf.modelspace()

## In the block below there will be created a dxf-File for analysis purposes.


In [5]:
#Read pillar circles from architecture wall
architecture_polylines = []
architecture_lines = []
fassaden_polylines = arichtecture_model.query("LWPolyline[layer=='A_05_FASSADE_ELEMENTE']")#.first.get_points('xy')

#lines
fassaden_lines = arichtecture_model.query("LINE[layer=='A_05_FASSADE_ELEMENTE']")

#Arcs
fassaden_arc = arichtecture_model.query("ARC[layer=='A_05_FASSADE_ELEMENTE']")

for polyline in fassaden_polylines:
    architecture_polylines.append(polyline)
for line in fassaden_lines:
    architecture_lines.append(line)

doc = ezdxf.new('R2000')
msp = doc.modelspace()
doc.layers.new(name='FASSADE_LINES', dxfattribs={'linetype': 'DASHED', 'color': 1})
doc.layers.new(name='FASSADE_POLYLINES', dxfattribs={'linetype': 'DASHED', 'color': 2})
doc.layers.new(name='FASSADE_ARCS', dxfattribs={'linetype': 'DASHED', 'color': 3})

#linetypes = ['BYLAYER', 'HIDDEN', '4.102_EG$0$CONTINOUS']
linetypes = ['HIDDEN']
for x in architecture_polylines:
    #mylinetype = x.dxf.linetype
    #if mylinetype not in linetypes:
    #    linetypes.append(mylinetype)
    if x.dxf.linetype in linetypes:
        msp.add_lwpolyline(x, dxfattribs={'layer': 'FASSADE_POLYLINES'})

#add lines to drawing
coordinates = []

for e in architecture_lines:
    #mylinetype = e.dxf.linetype
    #if mylinetype not in linetypes:
    #    linetypes.append(mylinetype)
    if e.dxftype() == 'LINE' and e.dxf.linetype in linetypes:
        tupel = (e.dxf.start, e.dxf.end)
        coordinates.append(tupel)

for x in coordinates:
    msp.add_line(x[0], x[1], dxfattribs={'layer': 'FASSADE_LINES'})


#add arcs to drwawing
for x in fassaden_arc:
    #mylinetype = x.dxf.linetype
    #if mylinetype not in linetypes:
    #    linetypes.append(mylinetype)
    if  x.dxf.linetype in linetypes:
        msp.add_arc(x.dxf.center, x.dxf.radius, x.dxf.start_angle, x.dxf.end_angle, True, dxfattribs={'layer': 'FASSADE_ARCS'})

print(linetypes)
doc.saveas("facade_dashed_2og.dxf")


['HIDDEN']


In [20]:
''' This method gets the lenght between two points.'''
def get_length_between_points(point_1: Point, point_2: Point) -> float:
    line_string = LineString([(point_1.x_coordinate, point_1.y_coordinate), (point_2.x_coordinate, point_2.y_coordinate)])
    return line_string.length

In [21]:
''' This method creates a linestring form two coordinates.'''
def get_line_string_of_two_coordinates(coordinate_1, coordinate_2):
    return LineString([(coordinate_1[0], coordinate_1[1]), (coordinate_2[0], coordinate_2[1])])

In [22]:
''' This method creates a point object from an coordinate tuple.'''
def create_point_from_coordinate(coordinate: tuple) -> Point:
    return Point(x_coordinate=coordinate[0], y_coordinate=coordinate[1])

In [24]:
''' This method creates a linestring from two point objects.'''
def create_line_string_of_two_points(point_1: Point, point_2: Point)-> LineString:
    return LineString([(point_1.x_coordinate, point_1.y_coordinate), (point_2.x_coordinate, point_2.y_coordinate)])

### The follwoing block is for extracting a simple plan for testing purposes.

In [25]:
#Simple plan auslesen
dxf_file = ezdxf.readfile(file_path_simple + file_name_simple)
dxf_extraction_simple = DxfExtractionSimple()
simple_polygons, polygons_with_virtual_entities, extracted_functionalities = dxf_extraction_simple.get_all_polygons(dxf_file ,file_path_simple + file_name_simple)


2021-06-11 00:02:34.598 | SUCCESS  | app.main.services.orientation_extraction:get_north_orientation:117 - Compass extraction successful, return coordinate system
2021-06-11 00:02:34.599 | INFO     | app.main.services.dxf_extraction_simple:get_all_polygons:78 - Orientation will be calculated.
2021-06-11 00:02:34.599 | SUCCESS  | app.main.services.extraction_helper:get_modelspace:190 - sucessfully get_modelspace() from dxf_file: <ezdxf.document.Drawing object at 0x1377ec6d0>
2021-06-11 00:02:34.600 | DEBUG    | app.main.services.extraction_helper:get_outline_polygon:199 - get_outline_polygon for modelspace: <ezdxf.layouts.layout.Modelspace object at 0x13affeca0>
2021-06-11 00:02:34.637 | DEBUG    | app.main.services.dxf_extraction_simple:get_all_polygons:91 - got all room_polylines on layer: RAUMPOLYGON
2021-06-11 00:02:35.058 | INFO     | app.main.services.dxf_extraction_simple:get_all_polygons:128 - ####### Functionality outer walls & neighbours #######
2021-06-11 00:02:35.058 | INFO  

In [30]:
''' This method creates a Line object from two coordinates.'''
def get_line_from_coordinates(coordinate_1, coordinate_2):
    line_string = get_line_string_of_two_coordinates(coordinate_1, coordinate_2)
    start_point = create_point_from_coordinate(coordinate_1)
    end_point = create_point_from_coordinate(coordinate_2)
    return Line("id", line_string, start_point=start_point, end_point=end_point)

''' This method creates a list of lines from a polyline. '''
def create_lines_from_polylines(polylines: list) ->list[Line]:
    lines = []
    for polyline in polylines:
        for i in range(len(polyline)-1):
            line = get_line_from_coordinates(polyline[i], polyline[i + 1])
            lines.append(line)
    return lines


''' This method creates a linestring from a wall object.'''
def create_line_string_for_wall(wall: Wall):
    return LineString([(wall.start_point.x_coordinate, wall.start_point.y_coordinate), (wall.end_point.x_coordinate, wall.end_point.y_coordinate)])


''' This method checks if a point is on a given linestring including a threshold value.'''
def is_point_on_line(point: Point, line: LineString):
    threshold = 1.2
    coordinate = ShapelyPoint(point.x_coordinate, point.y_coordinate)
    return line.distance(coordinate) < threshold
    

In [27]:
''' This method gets the window lenght from an given wall.'''
def get_window_length_of_wall(wall: Wall, lines: list[Line])->float:
    wall_line_string = create_line_string_for_wall(wall)
    window_length_sum = 0.0
    for line in lines:
        window_length = 0.0
        if not is_point_on_line(wall.start_point, line.line_string) and not is_point_on_line(wall.end_point, line.line_string):
            if is_point_on_line(line.start_point, wall_line_string) and is_point_on_line(line.end_point, wall_line_string):
                window_length = get_length_between_points(line.start_point, line.end_point)
                #print("Länge berechnet! Option 1")
           
        elif is_point_on_line(wall.start_point, line.line_string) and is_point_on_line(wall.end_point, line.line_string):
            window_length = get_length_between_points(wall.start_point, wall.end_point)
            #print("Länge berechnet! Option 2")
        else:
            wall_point_on_line = None
            line_point_on_wall = None
            if is_point_on_line(wall.start_point, line.line_string):
                wall_point_on_line = wall.start_point
            elif is_point_on_line(wall.end_point, line.line_string):
                wall_point_on_line = wall.end_point
        
            if is_point_on_line(line.start_point, wall_line_string):
                line_point_on_wall = line.start_point
            elif is_point_on_line(line.end_point, wall_line_string):
                line_point_on_wall = line.end_point
            
            if wall_point_on_line and line_point_on_wall:
                window_length = get_length_between_points(wall_point_on_line, line_point_on_wall)
                #print("Länge berechnet! Option 3")
        
        window_length_sum += window_length
    return window_length_sum



### The following block is used for testing the logic above. @Nico

In [31]:
facade_polylines = arichtecture_model.query("LWPolyline[layer=='A_05_FASSADE_ELEMENTE']")#.first.get_points('xy')
lines = create_lines_from_polylines(facade_polylines)
#lines
facade_lines = arichtecture_model.query("LINE[layer=='A_05_FASSADE_ELEMENTE']")
for facade_line in facade_lines:
    line = get_line_from_coordinates(facade_line.dxf.start, facade_line.dxf.end)
    lines.append(line)



print("Amount of lines found in facade layer: {}".format(str(len(lines))))

amount_walls = 0
for polygon in simple_polygons:
    for wall in polygon.walls:
        if not wall.is_outer_wall:
            continue
        amount_walls += 1

print("Amount of walls found in simple plan: {}".format(str(amount_walls)))
#loop durch alle polygons
#loop durch alle walls
#window_length berechnen
window_lengths = []
counter = 0
for polygon in simple_polygons:
    for wall in polygon.walls:
        if not wall.is_outer_wall:
            continue
        progress = round((counter / amount_walls) * 100, 2)
        if progress % 5 == 0:
            print("Progress: {} %".format(str(progress)))
        window_length = get_window_length_of_wall(wall, lines)
        wall.window_length = window_length
        window_lengths.append(window_length)
        counter += 1

print(window_lengths)



#minimale window length, maximale window length, wieviele haben length

Amount of lines found in facade layer: 5188
Amount of walls found in simple plan: 510
Progress: 0.0 %
Länge berechnet! Option 2
Länge berechnet! Option 3
Länge berechnet! Option 3
Länge berechnet! Option 3
Länge berechnet! Option 3
Länge berechnet! Option 3
Länge berechnet! Option 3
Länge berechnet! Option 3
Länge berechnet! Option 3
Länge berechnet! Option 3
Länge berechnet! Option 3
Länge berechnet! Option 3
Länge berechnet! Option 1
Länge berechnet! Option 1
Länge berechnet! Option 1
Länge berechnet! Option 1
Länge berechnet! Option 1
Länge berechnet! Option 1
Länge berechnet! Option 1
Länge berechnet! Option 1
Länge berechnet! Option 1
Länge berechnet! Option 1
Länge berechnet! Option 1
Länge berechnet! Option 1
Länge berechnet! Option 1
Länge berechnet! Option 1
Länge berechnet! Option 1
Länge berechnet! Option 1
Länge berechnet! Option 1
Länge berechnet! Option 1
Länge berechnet! Option 1
Länge berechnet! Option 1
Länge berechnet! Option 1
Länge berechnet! Option 1
Länge berechne

In [29]:
#write simple plan to architecture plan (just polygons)
doc = architecture_dxf
new_msp = doc.modelspace()
doc.layers.new(name='SIMPLE_PLAN', dxfattribs={'linetype': 'DASHED', 'color': 10})
polylines = simple_model.query(
            'LWPolyline[layer=="{}"]'.format("RAUMPOLYGON"))
for polyline in polylines:
    new_msp.add_lwpolyline(polyline, dxfattribs={'layer': 'RAUMPOLYGON'})
doc.saveas("architecture_simple.dxf")